In [1]:
import fairseq
from avhubert import hubert_pretraining, hubert
from avhubert import extract_roi

import cv2
import tempfile
from argparse import Namespace
import fairseq
import sys
import pickle
import joblib
from fairseq import checkpoint_utils, options, tasks, utils
from fairseq.dataclass.configs import GenerationConfig
import torch
import os
from os.path import exists
import subprocess
from avhubert import extract_roi
import extract_mouth

In [2]:
def newPredict(video_path, audio_path, user_dir, models, cfg, task):
  num_frames = int(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FRAME_COUNT))
  data_dir = tempfile.mkdtemp()
  tsv_cont = ["/\n", f"test-0\t{video_path}\t{audio_path}\t{num_frames}\t{int(16_000*num_frames/25)}\n"]
  label_cont = ["DUMMY\n"]
  with open(f"{data_dir}/test.tsv", "w") as fo:
    fo.write("".join(tsv_cont))
  with open(f"{data_dir}/test.wrd", "w") as fo:
    fo.write("".join(label_cont))
  utils.import_user_module(Namespace(user_dir=user_dir))
  modalities = ["audio", "video"]
  gen_subset = "test"
  gen_cfg = GenerationConfig(beam=20)
  cfg.task.modalities = modalities
  cfg.task.data = data_dir
  cfg.task.label_dir = data_dir
  cfg.task.max_sample_size = 1000
  task = tasks.setup_task(cfg.task)
  task.cfg.noise_wav = None # TODO: edit
  task.cfg.noise_prob = 0 # TODO: edit
  task.load_dataset(gen_subset, task_cfg=cfg.task)

  with torch.no_grad():
    mods = []
    for model in models:
      mods.append(model.cuda().eval())
      # model.cpu()
      torch.cuda.empty_cache()
  models = mods
  print(models)

  # models = [model.eval().cuda() for model in models]

  generator = task.build_generator(models, gen_cfg)

  def decode_fn(x):
      dictionary = task.target_dictionary
      symbols_ignore = generator.symbols_to_strip_from_output
      symbols_ignore.add(dictionary.pad())
      return task.datasets[gen_subset].label_processors[0].decode(x, symbols_ignore)

  itr = task.get_batch_iterator(dataset=task.dataset(gen_subset)).next_epoch_itr(shuffle=False)
  sample = next(itr)
  sample = utils.move_to_cuda(sample)
  hypos = task.inference_step(generator, models, sample)
  ref = decode_fn(sample['target'][0].int().cpu())
  hypo = hypos[0][0]['tokens'].int().cpu()
  hypo = decode_fn(hypo)
  return hypo

In [3]:
wd =os.getcwd()

In [5]:
# def load_model(wd, ckpt_path):
#     if not (exists(f'{wd}/data/model/models.pkl') and exists(f'{wd}/data/model/cfg.pkl') and exists(f'{wd}/data/model/task.pkl')):
#         models, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([ckpt_path])
#         with open(f'{wd}/data/model/models.pkl', 'wb') as file:
#             pickle.dump(models, file)
#         with open(f'{wd}/data/model/cfg.pkl', 'wb') as file:
#             pickle.dump(cfg, file)
#         with open(f'{wd}/task.pkl', 'wb') as file:
#             pickle.dump(task, file)
#     else:
#         with open('models.pkl', 'rb') as file:
#             models = pickle.load(file)
#         with open('cfg.pkl', 'rb') as file:
#             cfg = pickle.load(file)
#         with open('task.pkl', 'rb') as file:
#             task = pickle.load(file)

#     return models, cfg, task


ckpt_path = f"{wd}/data/misc/large_noise_pt_noise_ft_433h.pt"
# models, cfg, task = load_model(wd, ckpt_path)
models, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([ckpt_path])

: 

: 

In [4]:
ckpt_path = f"{wd}/data/misc/large_noise_pt_noise_ft_433h.pt"
models, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([ckpt_path])

In [5]:
print(wd)

/home/harveyw/harveyw/iw/av_hubert_updated/av_hubert/av_hubert


In [5]:
file_name = "angel_clip.mp4"

In [10]:
import cProfile
import re
cProfile.run("extract_roi.preprocess_video(compress_vid_path, '/' + mouth_roi_path, face_predictor_path, mean_face_path)")

100%|██████████| 278/278 [00:56<00:00,  4.88it/s]


         185755 function calls (181877 primitive calls) in 64.313 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      557    0.001    0.000    0.013    0.000 <__array_function__ internals>:2(all)
      278    0.000    0.000    0.007    0.000 <__array_function__ internals>:2(around)
      278    0.000    0.000    0.006    0.000 <__array_function__ internals>:2(atleast_2d)
      278    0.001    0.000    0.007    0.000 <__array_function__ internals>:2(atleast_3d)
      278    0.001    0.000    0.101    0.000 <__array_function__ internals>:2(clip)
      556    0.002    0.000    0.103    0.000 <__array_function__ internals>:2(concatenate)
      278    0.001    0.000    0.005    0.000 <__array_function__ internals>:2(copy)
      545    0.001    0.000    0.004    0.000 <__array_function__ internals>:2(copyto)
      267    0.001    0.000    0.011    0.000 <__array_function__ internals>:2(count_nonzero)
      267    0.001    0.000

In [7]:
orig_vid_path = wd + f'/data/video-orig/{file_name}'

updated_file_name = file_name.replace(".", "_")

compress_vid_path = wd + f'/data/video-comp/{updated_file_name}_compressed.mp4'
mouth_roi_path = wd[1:] + f'/data/video-roi/{updated_file_name}_roi.mp4'
audio_path = wd + f'/data/audio/{updated_file_name}_audio.wav'

# file_name = orig_vid_path[orig_vid_path.rfind('/') + 1:]

if not exists('/' + mouth_roi_path):
  command = f'ffmpeg -i {orig_vid_path} -vcodec libx265 -crf 28 -r 25 -vf scale=850:480 -ac 1 {compress_vid_path}'
  subprocess.call(command, shell=True)

  # EXTRACT ROI
  face_predictor_path = f"{wd}/data/misc/shape_predictor_68_face_landmarks.dat"
  mean_face_path = f"{wd}/data/misc/20words_mean_face.npy"
  extract_roi.preprocess_video(compress_vid_path, '/' + mouth_roi_path, face_predictor_path, mean_face_path)
  # extract_mouth.process_video(compress_vid_path, '/' + mouth_roi_path, face_predictor_path)

  # EXTRACT AUDIO
  command = f"ffmpeg -i {compress_vid_path} -ar 16000 -ac 1 -f wav {audio_path}"
  subprocess.call(command, shell=True)

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [8]:
user_dir = "./"
newHypo = newPredict(mouth_roi_path, audio_path, user_dir, models, cfg, task)
print(newHypo)

with open(f'{wd}/data/transcriptions/{updated_file_name}.txt', 'w') as f:
  f.write(f'{file_name} - Speech Transcription\n')
  f.write(newHypo)

{'_name': 'av_hubert_pretraining', 'data': '/tmp/tmpfqu0m_ec', 'labels': ['wrd'], 'label_dir': '/tmp/tmpfqu0m_ec', 'label_rate': -1, 'sample_rate': 16000, 'normalize': True, 'enable_padding': False, 'max_sample_size': 1000, 'min_sample_size': None, 'max_trim_sample_size': '${task.max_sample_size}', 'single_target': True, 'random_crop': False, 'pad_audio': True, 'pdb': False, 'stack_order_audio': 4, 'skip_verify': False, 'image_aug': True, 'image_crop_size': 88, 'image_mean': 0.421, 'image_std': 0.165, 'modalities': ['audio', 'video'], 'is_s2s': True, 'tokenizer_bpe_name': 'sentencepiece', 'tokenizer_bpe_model': '/checkpoint/bshi/data/lrs3//lang/spm/spm_unigram1000.model', 'noise_wav': None, 'noise_prob': 0.0, 'noise_snr': '0', 'noise_num': 1, 'fine_tuning': True}
self.cfg.noise_wav:  None
min keep: None
max keep: 1000
manifest path: /tmp/tmpfqu0m_ec/test.tsv
root:  /
names: [('home/harveyw/harveyw/iw/av_hubert_updated/av_hubert/av_hubert/data/video-roi/angel_clip_mp4_roi.mp4', '/home/h